In [2]:
from setup import * #gets the real data

/media/ryan/5e52f95b-dbda-4605-bec6-10d513d58772/GitHubProjExt/effective-xgboost-notes/setup.py:32: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,42,44,54,56,64,81,83,85,87,105,107,109,121,123,125,148,150,157,172,174,192,194,210,218,219,221,223,246,247,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,302,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
/media/ryan/5e52f95b-dbda-4605-bec6-10d513d58772/GitHubProjExt/effective-xgboost-notes/setup.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
/media/ryan/5e52f95b-dbda-4605-bec6-10d513d58772/GitHubProjExt/effective-xgboost-notes/setup.py:78: FutureWarning: Downcasting behavior in `replace` is d

In [5]:
import numpy as np
import numpy.random as rn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import tree, dummy, preprocessing, ensemble
from sklearn.metrics import accuracy_score, roc_auc_score
import xgboost as xgb
import dtreeviz
import yellowbrick.model_selection as ms
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, pyll
from typing import Any, Dict, Union, Sequence
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
#also, must install graphviz for plotting to work with xgb

In [19]:
params={'random_state': 42}
rounds = [{'max_depth': hp.quniform('max_depth', 1, 8, 1), #tree
    'min_child_weight': hp.loguniform('min_child_weight',-2,3)},
    {'subsample': hp.uniform('subsample', 0.5, 1), #stochastic
    'colsample_bytree': hp.uniform('colsample_bytree', .5, 1)},
    {'reg_alpha': hp.uniform('reg_alpha', 0, 10),
    'reg_lambda': hp.uniform('reg_lambda',1,10)},
    {'gamma': hp.loguniform('gamma', -10, 10), #regularization
    'learning_rate': hp.loguniform('learning_rate',-7,0), #boosting
} ] #note these are grouped into dictionaries


In [20]:
def hyperparameter_tuning(space: Dict[str, Union[float, int]],\
    X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame,\
    y_test: pd.Series, early_stopping_rounds: int=50, metric:callable=accuracy_score) -> Dict[str, Any]:
    """
    Function for tuning hyperparameters. This is the wrong way to do it! Just a grid search/stepwise. 
    A better way is done in chapter 13. The better way takes a fraction of the time for the same parameters!
    """
    int_vals = ['max_depth','reg_alpha']
    space={k: (int(val) if k in int_vals else val) for k, val in space.items()}
    space['early_stopping_rounds'] = early_stopping_rounds
    model = xgb.XGBClassifier(**space)
    evaluation = [(X_train, y_train), (X_test, y_test)]
    model.fit(X_train, y_train, eval_set=evaluation, verbose=False)
    pred = model.predict(X_test)
    score = metric(y_test, pred)
    return {'loss': -score, 'status': STATUS_OK, 'model':model}

In [54]:
all_trials = []
counter = 0
param_dict = dict()
for round in rounds:
    params = {**params, **round} #putting these two together has the ones on the right overwrite the ones on the left!
    trials = Trials()
    best = fmin(fn=lambda space: hyperparameter_tuning(space, X_train, y_train, X_test, y_test), space=params, \
    algo=tpe.suggest, max_evals=20, trials=trials, timeout=60*5 ) #put max_evals way down. that's all.
    params = {**params, **best} #each round, update params with the best so far from this round
    param_dict[counter] = params
    counter += 1
    all_trials.append(trials)

# in the end, the params that comes out is the best params across all rounds!

100%|██████████| 20/20 [00:03<00:00,  5.57trial/s, best loss: -0.7624309392265194]


In [55]:
param_dict

{0: {'random_state': 42,
  'max_depth': 4.0,
  'min_child_weight': 2.458212305327815,
  'subsample': 0.5759675916879781,
  'colsample_bytree': 0.6700438134020338,
  'reg_alpha': 3.2912936851132777,
  'reg_lambda': 1.6849564284757554,
  'gamma': 0.002619509439664793,
  'learning_rate': 0.27445215578940063},
 1: {'random_state': 42,
  'max_depth': 4.0,
  'min_child_weight': 2.458212305327815,
  'subsample': 0.8892271693608613,
  'colsample_bytree': 0.9301432941452263,
  'reg_alpha': 3.2912936851132777,
  'reg_lambda': 1.6849564284757554,
  'gamma': 0.002619509439664793,
  'learning_rate': 0.27445215578940063},
 2: {'random_state': 42,
  'max_depth': 4.0,
  'min_child_weight': 2.458212305327815,
  'subsample': 0.8892271693608613,
  'colsample_bytree': 0.9301432941452263,
  'reg_alpha': 0.8990193851150607,
  'reg_lambda': 3.5050157401650095,
  'gamma': 0.002619509439664793,
  'learning_rate': 0.27445215578940063},
 3: {'random_state': 42,
  'max_depth': 4.0,
  'min_child_weight': 2.4582123

In [58]:
{**params,**round} #putting these two together has the ones on the right overwrite the ones on the left!

{'random_state': 42,
 'max_depth': 4.0,
 'min_child_weight': 2.458212305327815,
 'subsample': 0.8892271693608613,
 'colsample_bytree': 0.9301432941452263,
 'reg_alpha': 0.8990193851150607,
 'reg_lambda': 3.5050157401650095,
 'gamma': <hyperopt.pyll.base.Apply at 0x7f55006c0c10>,
 'learning_rate': <hyperopt.pyll.base.Apply at 0x7f55006c1590>}

In [ ]:
# above is the same as the following, except this does the change in place and that creates a new one.
#for items in round:
#    params[item] = round[item]

In [62]:
{**params}

{'random_state': 42,
 'max_depth': 4.0,
 'min_child_weight': 2.458212305327815,
 'subsample': 0.8892271693608613,
 'colsample_bytree': 0.9301432941452263,
 'reg_alpha': 0.8990193851150607,
 'reg_lambda': 3.5050157401650095,
 'gamma': 0.060498412496152204,
 'learning_rate': 0.16400037064490072}

In [60]:
{**round}

{'gamma': <hyperopt.pyll.base.Apply at 0x7f55006c0c10>,
 'learning_rate': <hyperopt.pyll.base.Apply at 0x7f55006c1590>}

In [23]:
def plot_3d_mesh(df: pd.DataFrame, x_col: str, y_col: str, z_col: str) -> go.Figure:
    fig = go.Figure(data=[go.Mesh3d(x=df[x_col], y=df[y_col], z=df[z_col], intensity=df[z_col]/ df[z_col].min(),
        hovertemplate=f"{z_col}: %{{z}}<br>{x_col}: %{{x}}<br>{y_col}: %{{y}}<extra></extra>")],
    )
    fig.update_layout(
        title=dict(text=f'{y_col} vs {x_col}'),
        scene=dict(
            xaxis_title=x_col,
            yaxis_title=y_col,
            zaxis_title=z_col),
        width=700,
        margin=dict(r=20, b=10, l=10, t=50)
    )
    return fig

In [24]:
def trial2df(trial: Sequence[Dict[str, Any]]) -> pd.DataFrame:
    """
    Convert a trial object (sequence of trial dictionaries) to a Pandas DataFrame.

    Parameters:
    trial: List[Dict[str, Any]]
        A list of trial dictionaries.

    Returns
    pd.DataFrame - A DataFrame with columns for the loss, trial id, and values from each trial dictionary.
    """
    vals = [] #will be a list of dictionaries, then a pd.DataFrame
    for t in trial:
        result = t['result']
        misc = t['misc']
        val = {k:(v[0] if isinstance(v, list) else v) for k, v in misc['vals'].items()
               } #dictionary.
        val['loss'] = result['loss']
        val['tid'] = t['tid']
        vals.append(val)
    return pd.DataFrame(vals)

In [25]:
plot_3d_mesh(trial2df(all_trials[2]),'reg_alpha','reg_lambda','loss')

In [37]:
all_trials[2].best_trial

{'state': 2,
 'tid': 4,
 'spec': None,
 'result': {'loss': -0.7624309392265194,
  'status': 'ok',
  'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=0.8896302716546785, device=None,
                early_stopping_rounds=50, enable_categorical=False,
                eval_metric=None, feature_types=None, gamma=None,
                grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=4, max_leaves=None,
                min_child_weight=15.207966409339303, missing=nan,
                monotone_constraints=None, multi_strategy=None, n_estimators=None,
                n_jobs=None, num_parallel_tree=None, random_state=42, ...)},
 'misc': {'tid': 4,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  '

In [33]:
x = all_trials[3]

In [42]:
for tri in all_trials:
    print(tri.best_trial)

{'state': 2, 'tid': 14, 'spec': None, 'result': {'loss': -0.7668508287292818, 'status': 'ok', 'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=15.207966409339303, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)}, 'misc': {'tid': 14, 'cmd': ('domain_attachment', 'FMinIter_Domain'), 'workdir': None, 'idxs': {'max_depth': [14], 'min_child_weigh

In [63]:
for tri in all_trials:
    print(f"loss: { tri.best_trial['result']['loss'] }, best_trial_result: {tri.best_trial['result']}")

loss: -0.7635359116022099, best_trial_result: {'loss': -0.7635359116022099, 'status': 'ok', 'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6700438134020338, device=None,
              early_stopping_rounds=50, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.002619509439664793,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.27445215578940063,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=2.458212305327815, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)}
loss: -0.7668508287292818, best_trial_result: {'loss': -0.7668508287292818,

## 13.3 Training with an Optimized Model

In [70]:
step_params = params
step_params['max_depth'] = int(step_params['max_depth'] )

In [71]:
xg_step = xgb.XGBClassifier(**step_params, early_stopping_rounds=50, n_estimators=500)

In [74]:
xg_step.fit(X_train, y_train, eval_set = [(X_train, y_train),(X_test,y_test)],verbose=100) 
#eval_set is a list of items to be evaluated. It is helpful during early stopping
#note, it reports validation_0-logloss and validation_1-logloss!
#validation_0 is against the training and validation_1 is against the test

[0]	validation_0-logloss:0.65409	validation_1-logloss:0.65424


[100]	validation_0-logloss:0.44758	validation_1-logloss:0.49611
[151]	validation_0-logloss:0.42877	validation_1-logloss:0.49876


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9301432941452263, device=None,
              early_stopping_rounds=50, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.060498412496152204,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.16400037064490072,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=2.458212305327815, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [76]:
xg_step.score(X_test, y_test)

0.7524861878453039

In [77]:
xg_def = xgb.XGBClassifier()
xg_def.fit(X_train, y_train)
xg_def.score(X_test, y_test)

0.7502762430939226

In [ ]:
# A small improvement